# W05A — Diseño relacional mínimo: **PK/FK**, llaves surrogate y “star schema” (DuckDB)

**Objetivo:** pasar de “join por hostname” a un modelo **fact/dim con llaves estables**:
- **PK** (Primary Key) para identificar filas,
- **FK** (Foreign Key) para asegurar **integridad referencial**,
- **Surrogate keys** para dimensiones (ej. `host_id`),
- Validación con **evidencia** (anti-join de huérfanos).

> Importante: en *analítica/warehouse* a veces no se “enfuerzan” constraints en producción.
> Aquí los usamos como **contrato ejecutable** + herramienta pedagógica.

## Bibliografía (W05A)

### DDIA (Kleppmann)
- **Cap. 2 — Data Models and Query Languages**
  - Relacional vs documento vs grafo (por qué SQL es útil cuando hay relaciones y joins).
  - Identificadores y relaciones (equivalentes a FK/referencias).

### Dimensional modeling (complementario)
- Kimball (surrogate keys en dimensiones y por qué ayudan en fact tables).

### DuckDB (práctica)
- Constraints: `PRIMARY KEY`, `FOREIGN KEY`, `UNIQUE`, `NOT NULL`

In [ ]:
from pathlib import Path
import duckdb

PROJECT_ROOT = Path(".").resolve()
RAW_DIR = PROJECT_ROOT / "data" / "raw"
DB_PATH = PROJECT_ROOT / "data" / "exoplanets.duckdb"
ART_DIR = PROJECT_ROOT / "artifacts"
DOCS_DIR = PROJECT_ROOT / "docs"

RAW_DIR.mkdir(parents=True, exist_ok=True)
ART_DIR.mkdir(parents=True, exist_ok=True)
DOCS_DIR.mkdir(parents=True, exist_ok=True)

con = duckdb.connect(str(DB_PATH))

raw_csv = RAW_DIR / "pscomppars.csv"
if not raw_csv.exists():
    raise FileNotFoundError(f"No encuentro {raw_csv}. Necesitas el CSV de W01/W02.")

def sql_quote(s: str) -> str:
    return "'" + s.replace("'", "''") + "'"

con.execute(f'''
CREATE OR REPLACE VIEW raw_ps AS
SELECT * FROM read_csv_auto({sql_quote(str(raw_csv.resolve()))})
''')

# Reconstrucción mínima (idéntica al espíritu de W03B) para que el notebook sea autocontenido
con.execute("DROP TABLE IF EXISTS silver_planet")
con.execute('''
CREATE TABLE silver_planet AS
SELECT
  pl_name,
  hostname,
  discoverymethod,
  disc_year,
  sy_snum,
  sy_pnum,
  sy_dist,
  ra,
  dec,
  pl_orbper,
  pl_rade,
  pl_bmasse,
  pl_eqt,
  st_teff,
  st_rad,
  st_mass
FROM raw_ps
WHERE pl_name IS NOT NULL
  AND hostname IS NOT NULL
  AND (disc_year IS NULL OR (disc_year BETWEEN 1980 AND 2026))
  AND (pl_rade  IS NULL OR (pl_rade  > 0 AND pl_rade  <= 30))
  AND (pl_bmasse IS NULL OR (pl_bmasse > 0))
''')

con.execute("DROP TABLE IF EXISTS dim_host_full")
con.execute('''
CREATE TABLE dim_host_full AS
SELECT
  hostname,
  MAX(sy_dist)  AS sy_dist,
  MAX(ra)       AS ra,
  MAX(dec)      AS dec,
  MAX(st_teff)  AS st_teff,
  MAX(st_rad)   AS st_rad,
  MAX(st_mass)  AS st_mass
FROM silver_planet
GROUP BY hostname
''')

con.execute("DROP TABLE IF EXISTS fact_planet")
con.execute('''
CREATE TABLE fact_planet AS
SELECT DISTINCT
  pl_name,
  hostname,
  discoverymethod,
  disc_year,
  pl_orbper,
  pl_rade,
  pl_bmasse,
  pl_eqt
FROM silver_planet
''')

con.sql("SELECT COUNT(*) AS n_fact, COUNT(DISTINCT pl_name) AS n_pl, COUNT(DISTINCT hostname) AS n_hosts FROM fact_planet").show()

## 1) Conceptos mínimos

### Grain (granularidad)
- `fact_planet`: **1 fila ~ 1 planeta** (identificado por `pl_name` en nuestro Core).
- `dim_host`: **1 fila ~ 1 estrella anfitriona** (identificada por `hostname`).

### Natural key vs Surrogate key
- **Natural key:** viene del dominio (ej. `hostname`).
- **Surrogate key:** entero generado (ej. `host_id`).
  - Ventajas típicas en warehouse: más pequeño, más estable para joins, más fácil para evolucionar modelos.

### PK/FK
- **PK**: “esta columna identifica una fila”.
- **FK**: “esta columna debe existir en la tabla padre”.

In [ ]:
# TODO 1 (estudiante): crea dim_host_sk igual que en clase
# - Debe tener host_id como PRIMARY KEY
# - hostname NOT NULL y UNIQUE
# - Inserta datos desde dim_host_full usando ROW_NUMBER()

con.execute("DROP TABLE IF EXISTS dim_host_sk")

# TODO: CREATE TABLE ...
# con.execute(""" ... """)

# TODO: INSERT ...
# con.execute(""" ... """)

# Validación:
con.sql("SELECT COUNT(*) AS n_rows, COUNT(DISTINCT hostname) AS n_keys FROM dim_host_sk").show()

In [ ]:
# TODO 2 (estudiante): crea fact_planet_sk con FK a dim_host_sk(host_id)
# - pl_name PRIMARY KEY
# - host_id NOT NULL REFERENCES dim_host_sk(host_id)
# - Inserta desde fact_planet JOIN dim_host_sk (por hostname)

con.execute("DROP TABLE IF EXISTS fact_planet_sk")

# TODO: CREATE TABLE ...
# con.execute(""" ... """)

# TODO: INSERT ...
# con.execute(""" ... """)

con.sql("SELECT COUNT(*) AS n_fact_sk FROM fact_planet_sk").show()

# Check huérfanos
con.sql('''
SELECT COUNT(*) AS orphan_rows
FROM fact_planet_sk f
LEFT JOIN dim_host_sk d
  ON f.host_id = d.host_id
WHERE d.host_id IS NULL
''').show()

In [ ]:
# TU TURNO 3: consulta analítica usando el modelo con llaves
# Objetivo: top 10 métodos de descubrimiento y n_planets

q = '''
SELECT
  discoverymethod,
  COUNT(*) AS n_planets
FROM fact_planet_sk
WHERE discoverymethod IS NOT NULL
GROUP BY discoverymethod
ORDER BY n_planets DESC
LIMIT 10
'''
con.sql(q).show()

## 2) Data Contract (actualización para H2)

En `docs/data_contract.md` debe quedar:
- **Datasets:** `dim_host_sk`, `fact_planet_sk`
- **Grain**:
  - `dim_host_sk`: 1 fila por `hostname`
  - `fact_planet_sk`: 1 fila por `pl_name`
- **Keys/Constraints**:
  - PK dim: `host_id` (y `hostname` UNIQUE)
  - PK fact: `pl_name`
  - FK fact → dim: `host_id`
- **Checks mínimos**:
  - `orphan_rows = 0`
  - `n_rows(dim) == n_keys(hostname)`

## Para entregar (W05A)

### En clase
1) Evidencia (pegar en tu bitácora o `docs/w05a_evidence.md`):
   - `SELECT COUNT(*) , COUNT(DISTINCT hostname)` de `dim_host_sk`
   - `orphan_rows` en `fact_planet_sk`
2) `docs/decisions_log.md`: 1 entrada:
   - “Usamos `host_id` surrogate key y FK” + evidencia (conteos + orphan_rows)

### Tarea (parte de H2)
1) Actualiza `docs/data_contract.md` con:
   - grain, keys, checks
2) `docs/w05a_modeling_notes.md` (máx 1 página):
   - Natural key vs surrogate (2–3 bullets)
   - Cuándo usarías FK (y cuándo preferirías validar con queries)

## H2 (fin de semana) — recordatorio (Core)

Al final de W05 (A+B) el proyecto debe tener, como mínimo:
- Pipeline reproducible: Raw/Bronze-lite → Silver → **dim/fact con llaves** → 2 outputs Gold
- `docs/data_contract.md` actualizado (grains + keys + checks)
- Evidencia en `artifacts/` (al menos 2 CSV/outputs) y `docs/decisions_log.md` con 2 decisiones clave